In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders.binary import BinaryEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Visualizing the data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TP2-OD/secondary_data.csv', sep=";")
df

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,g,o,f,e,NaN,w,16.95,...,s,y,w,u,w,t,g,NaN,d,w
1,p,16.60,x,g,o,f,e,NaN,w,17.99,...,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,x,g,o,f,e,NaN,w,17.80,...,s,y,w,u,w,t,g,NaN,d,w
3,p,14.17,f,h,e,f,e,NaN,w,15.77,...,s,y,w,u,w,t,p,NaN,d,w
4,p,14.64,x,h,o,f,e,NaN,w,16.53,...,s,y,w,u,w,t,p,NaN,d,w
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61064,p,1.18,s,s,y,f,f,f,f,3.93,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,a
61065,p,1.27,f,s,y,f,f,f,f,3.18,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,a
61066,p,1.27,s,s,y,f,f,f,f,3.86,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,u
61067,p,1.24,f,s,y,f,f,f,f,3.56,...,NaN,NaN,y,NaN,NaN,f,f,NaN,d,u


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  object 
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  object 
 3   cap-surface           46949 non-null  object 
 4   cap-color             61069 non-null  object 
 5   does-bruise-or-bleed  61069 non-null  object 
 6   gill-attachment       51185 non-null  object 
 7   gill-spacing          36006 non-null  object 
 8   gill-color            61069 non-null  object 
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-root             9531 non-null   object 
 12  stem-surface          22945 non-null  object 
 13  stem-color            61069 non-null  object 
 14  veil-type             3177 non-null   object 
 15  veil-color         

In [ ]:
df.groupby('cap-surface').count()

,class,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
cap-surface,,,,,,,,,,,,,,,,,,,,
d,4432,4432,4432,4432,4432,4432,2872,4432,4432,4432,785,1472,4432,0,0,4432,4265,79,4432,4432
e,2584,2584,2584,2584,2584,2118,1346,2584,2584,2584,708,0,2584,0,535,2584,2407,97,2584,2584
g,4724,4724,4724,4724,4724,3665,1765,4724,4724,4724,183,1794,4724,536,536,4724,4724,0,4724,4724
h,4974,4974,4974,4974,4974,3920,2812,4974,4974,4974,546,3247,4974,693,1352,4974,4620,645,4974,4974
i,2225,2225,2225,2225,2225,1519,878,2225,2225,2225,268,550,2225,0,524,2225,2225,1156,2225,2225
k,2303,2303,2303,2303,2303,1822,969,2303,2303,2303,94,662,2303,0,0,2303,2117,353,2303,2303
l,1412,1412,1412,1412,1412,1412,1059,1412,1412,1412,353,353,1412,0,0,1412,1412,353,1412,1412
s,7608,7608,7608,7608,7608,6912,5293,7608,7608,7608,1665,3427,7608,353,659,7608,7255,1407,7608,7608
t,8196,8196,8196,8196,8196,7189,5662,8196,8196,8196,493,3523,8196,536,1168,8196,7844,656,8196,8196


In [ ]:
df.groupby('gill-spacing').count()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
gill-spacing,,,,,,,,,,,,,,,,,,,,
c,24710,24710,24710,18709,24710,24710,20474,24710,24710,24710,3177,9531,24710,1059,2824,24710,23651,2118,24710,24710
d,7766,7766,7766,7060,7766,7766,7060,7766,7766,7766,706,3883,7766,0,0,7766,7413,706,7766,7766
f,3530,3530,3530,1765,3530,3530,3530,3530,3530,3530,706,1765,3530,0,0,3530,3530,353,3530,3530


In [ ]:
df[['veil-type','class']].groupby('veil-type').count()

,class
veil-type,
u,3177


In [ ]:
df_test = df[['class','veil-type']]
df_test['veil-type'] = df['veil-type'].fillna('?')
df_test[df_test['veil-type'] == '?'].groupby('class').count()

<ipython-input-198-197fce58cd0d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['veil-type'] = df['veil-type'].fillna('?')


,veil-type
class,
e,26122
p,31770


In [ ]:
df_test = df[['class','spore-print-color']]
df_test['spore-print-color'] = df['spore-print-color'].fillna('?')
df_test[df_test['spore-print-color'] == '?'].groupby('class').count()

<ipython-input-199-aceb46ec7838>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['spore-print-color'] = df['spore-print-color'].fillna('?')


,spore-print-color
class,
e,25416
p,29299


In [ ]:
df[['stem-root','class']].groupby('stem-root').count()

,class
stem-root,
b,3177
c,706
f,1059
r,1412
s,3177


In [ ]:
df_test = df[['class','stem-root']]
df_test['stem-root'] = df['stem-root'].fillna('?')
df_test[df_test['stem-root'] == '?'].groupby('class').count()

<ipython-input-201-9b770bbc2b1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['stem-root'] = df['stem-root'].fillna('?')


,stem-root
class,
e,23651
p,27887


In [ ]:
df_test = df[['class','stem-surface']]
df_test['stem-surface'] = df['stem-surface'].fillna('?')
df_test[df_test['stem-surface'] == '?'].groupby('class').count()

<ipython-input-202-d6c93b35cd9f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['stem-surface'] = df['stem-surface'].fillna('?')


,stem-surface
class,
e,18709
p,19415


In [ ]:
df1 = df.drop(columns=['spore-print-color','veil-color'], axis=1)

In [ ]:
df1['class'] = df1['class'].map(lambda x: 1 if x == 'p' else 0)

In [ ]:
X = df1.drop('class', axis=1)
y = df1['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [ ]:
X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.5, random_state=7)

##Feature Engineering


In [ ]:
X_test['cap-surface'] = X_test['cap-surface'].fillna('?')
X_train['cap-surface'] = X_train['cap-surface'].fillna('?')
X_validation['cap-surface'] = X_validation['cap-surface'].fillna('?')

In [ ]:
X_test['gill-attachment'] = X_test['gill-attachment'].fillna('?')
X_train['gill-attachment'] = X_train['gill-attachment'].fillna('?')
X_validation['gill-attachment'] = X_validation['gill-attachment'].fillna('?')

In [ ]:
X_test['gill-spacing'] = X_test['gill-spacing'].fillna(method='ffill')
X_train['gill-spacing'] = X_train['gill-spacing'].fillna(method='ffill')
X_validation['gill-spacing'] = X_validation['gill-spacing'].fillna(method='ffill')

In [ ]:
X_test['stem-root'] = X_test['stem-root'].fillna(method='ffill')
X_train['stem-root'] = X_train['stem-root'].fillna(method='ffill')
X_validation['stem-root'] = X_validation['stem-root'].fillna(method='ffill')
#Hay casos que los sets tienen los primeros registros como NaNs
X_test['stem-root'] = X_test['stem-root'].fillna(X_test['stem-root'].mode().values[0])
X_train['stem-root'] = X_train['stem-root'].fillna(X_train['stem-root'].mode().values[0])
X_validation['stem-root'] = X_validation['stem-root'].fillna(X_validation['stem-root'].mode().values[0])

In [ ]:
X_test['stem-surface'] = X_test['stem-surface'].fillna(method='ffill')
X_train['stem-surface'] = X_train['stem-surface'].fillna(method='ffill')
X_validation['stem-surface'] = X_validation['stem-surface'].fillna(method='ffill')
#Hay casos que los sets tienen los primeros registros como NaNs
X_test['stem-surface'] = X_test['stem-surface'].fillna(X_test['stem-surface'].mode().values[0])
X_train['stem-surface'] = X_train['stem-surface'].fillna(X_train['stem-surface'].mode().values[0])
X_validation['stem-surface'] = X_validation['stem-surface'].fillna(X_validation['stem-surface'].mode().values[0])

In [ ]:
X_test['ring-type'] = X_test['ring-type'].fillna('?')
X_train['ring-type'] = X_train['ring-type'].fillna('?')
X_validation['ring-type'] = X_validation['ring-type'].fillna('?')

In [ ]:
X_test.loc[X_test['has-ring'] == 't', 'veil-type'] = X_test.loc[X_test['has-ring'] == 't', 'veil-type'].fillna('none')
X_train.loc[X_train['has-ring'] == 't', 'veil-type'] = X_train.loc[X_train['has-ring'] == 't', 'veil-type'].fillna('none')
X_validation.loc[X_validation['has-ring'] == 't', 'veil-type'] = X_validation.loc[X_validation['has-ring'] == 't', 'veil-type'].fillna('none')

In [ ]:
X_test.loc[X_test['has-ring'] == 'f', 'veil-type'] = X_test.loc[X_test['has-ring'] == 'f', 'veil-type'].fillna('p')
X_train.loc[X_train['has-ring'] == 'f', 'veil-type'] = X_train.loc[X_train['has-ring'] == 'f', 'veil-type'].fillna('p')
X_validation.loc[X_validation['has-ring'] == 'f', 'veil-type'] = X_validation.loc[X_validation['has-ring'] == 'f', 'veil-type'].fillna('p')

In [ ]:
columns_to_mean = ['cap-color','gill-color','stem-color','ring-type', 'stem-surface', 'cap-surface']
mean_encoders_dict = {}

In [ ]:
for column in columns_to_mean:
  mean_encoder = TargetEncoder()

  encoded_col_train = mean_encoder.fit_transform(X_train[[column]], y_train)
  encoded_col_test = mean_encoder.transform(X_test[[column]], y_test)
  encoded_col_validation = mean_encoder.transform(X_validation[[column]], y_validation)

  mean_encoders_dict[column] = mean_encoder

  X_train[column] = encoded_col_train
  X_test[column] = encoded_col_test
  X_validation[column] = encoded_col_validation

In [ ]:
columns_to_ohe = ['cap-shape', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'stem-root', 'veil-type', 'has-ring', 'season', 'habitat']
ohe_dict = {}

In [ ]:
for column in columns_to_ohe:
  ohe = OneHotEncoder(drop='first')

  encoded_col_train = ohe.fit_transform(X_train[[column]]).todense().astype(int)
  encoded_col_test = ohe.transform(X_test[[column]]).todense().astype(int)
  encoded_col_validation = ohe.transform(X_validation[[column]]).todense().astype(int)

  columns = ohe.get_feature_names_out([column])
  ohe_dict[column] = ohe

  X_train = X_train.reset_index().drop(['index',column], axis=1).join(pd.DataFrame(encoded_col_train,columns=columns))
  X_test = X_test.reset_index().drop(['index',column], axis=1).join(pd.DataFrame(encoded_col_test,columns=columns))
  X_validation = X_validation.reset_index().drop(['index',column], axis=1).join(pd.DataFrame(encoded_col_validation,columns=columns))

In [ ]:
X_train['stem-width'] = X_train['stem-width'].map(lambda x: x/10)
X_test['stem-width'] = X_test['stem-width'].map(lambda x: x/10)
X_validation['stem-width'] = X_validation['stem-width'].map(lambda x: x/10)

##Tunning


In [ ]:
xgboost = GradientBoostingClassifier()

In [ ]:
params = {
    'loss' : ['log_loss','deviance','exponential'],
    'learning_rate': [0.01,0.05,0.1,0.5,1,5,10,50,100],
    'n_estimators' : [25,50,75,100,150,200,300],
    'subsample' : [0.75,0.8,0.85,0.9,0.95,1],
    'min_samples_split' : [5,7,10,12,15,20,25,50,75],
    'min_samples_leaf' : [1,3,5,8,10,12,15,20,25,30],
    'max_features' : ['sqrt','log2'],
    'max_depth': [4,5,6,7,8,9,10] #consideramos valores más chicos que para Random Forest
}
rs = RandomizedSearchCV(estimator=xgboost, param_distributions=params, cv=5, n_iter=200, scoring='roc_auc', random_state=7)
rs.fit(X_train, y_train)

In [ ]:
rs.best_params_

{'subsample': 0.75,
 'n_estimators': 200,
 'min_samples_split': 20,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 8,
 'loss': 'deviance',
 'learning_rate': 0.5}

In [ ]:
rs.best_score_

1.0

In [ ]:
model = rs.best_estimator_
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


GradientBoostingClassifier(learning_rate=0.5, loss='deviance', max_depth=8,
                           max_features='sqrt', min_samples_split=20,
                           n_estimators=200, subsample=0.75)

##Testing and Validation

In [ ]:
#Score de validación
roc_auc_score(y_validation, model.predict_proba(X_validation)[:, 1])

0.9999995656555126

Pienso que se logro el mismo valor que para Random Forest porque en ambos casos la búsqueda de los híper parámetros se realizo teniendo varios en cuenta, con distintos valores posibles e iterando múltiples veces. La búsqueda, aunque no haya sido Grid Search, fue extensa igualmente.

In [ ]:
y_pred = model.predict(X_test)
y_test = y_test.astype(int)

In [ ]:
#Score en test
accuracy_score(y_test, y_pred)

1.0

In [ ]:
y_pred = y_pred.tolist()
pred_sr = pd.Series(y_pred)

In [ ]:
pred_sr.to_csv('/content/drive/MyDrive/TP2-OD/predicciones_parte_III.csv')

In [ ]:
roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

1.0